In [75]:
import pandas as pd
import duckdb as db

In [76]:
# Telecharger les données

sales = pd.read_csv("c:/Users/Moi/sql_challenge/database/sales.csv")
sales.head()

,sale_day,sale_month,sale_time,branch,article,quantity,revenue
0,2021-08-11,AUG,11:00,New York,Rolex P1,1,3000.0
1,2021-08-14,AUG,11:20,New York,Rolex P1,2,6000.0
2,2021-08-17,AUG,10:00,Paris,Omega 100,3,4000.0
3,2021-08-19,AUG,10:00,London,Omega 100,1,1300.0
4,2021-07-17,JUL,09:30,Paris,Cartier A1,1,2000.0


In [83]:
sales.shape

(20, 7)

In [77]:
# requète qui permet d'obtenir la quantité d'articles vendues  sans la clause over()
query= """
SELECT  SUM(quantity)
FROM sales
"""
db.sql(query)

┌───────────────┐
│ sum(quantity) │
│    int128     │
├───────────────┤
│            34 │
└───────────────┘

In [81]:
# requète qui permet d'obtenir la quantité d'articles vendues  avec la clause over()
query= """
SELECT*, SUM(quantity) over()
FROM sales
"""
db.sql(query)

┌────────────┬────────────┬───────────┬──────────┬──────────────┬──────────┬─────────┬───────────────────────┐
│  sale_day  │ sale_month │ sale_time │  branch  │   article    │ quantity │ revenue │ sum(quantity) OVER () │
│  varchar   │  varchar   │  varchar  │ varchar  │   varchar    │  int64   │ double  │        int128         │
├────────────┼────────────┼───────────┼──────────┼──────────────┼──────────┼─────────┼───────────────────────┤
│ 2021-08-11 │ AUG        │ 11:00     │ New York │ Rolex P1     │        1 │  3000.0 │                    34 │
│ 2021-08-14 │ AUG        │ 11:20     │ New York │ Rolex P1     │        2 │  6000.0 │                    34 │
│ 2021-08-17 │ AUG        │ 10:00     │ Paris    │ Omega 100    │        3 │  4000.0 │                    34 │
│ 2021-08-19 │ AUG        │ 10:00     │ London   │ Omega 100    │        1 │  1300.0 │                    34 │
│ 2021-07-17 │ JUL        │ 09:30     │ Paris    │ Cartier A1   │        1 │  2000.0 │                    34 │
│

## La clause over() avec la sous-clause PARTITION BY pour les fonctions d'aggregrations

In [79]:
# une requète qui  qui renvoie la quantité totale d'unités vendues pour chaque article

query= """
SELECT*, SUM(quantity) OVER(PARTITION BY article)
FROM sales
"""
db.sql(query)

┌────────────┬────────────┬───────────┬──────────┬──────────────┬──────────┬─────────┬─────────────────────────────────┐
│  sale_day  │ sale_month │ sale_time │  branch  │   article    │ quantity │ revenue │ sum(quantity) OVER (PARTITION…  │
│  varchar   │  varchar   │  varchar  │ varchar  │   varchar    │  int64   │ double  │             int128              │
├────────────┼────────────┼───────────┼──────────┼──────────────┼──────────┼─────────┼─────────────────────────────────┤
│ 2021-08-11 │ AUG        │ 11:00     │ New York │ Rolex P1     │        1 │  3000.0 │                               6 │
│ 2021-08-14 │ AUG        │ 11:20     │ New York │ Rolex P1     │        2 │  6000.0 │                               6 │
│ 2021-08-02 │ AUG        │ 14:15     │ London   │ Rolex P1     │        1 │  3200.0 │                               6 │
│ 2021-08-30 │ AUG        │ 15:30     │ New York │ Rolex P1     │        1 │  3700.0 │                               6 │
│ 2021-09-03 │ SEP        │ 20:3

## Over avec la sous clause ORDER BY pour les fonctions d'aggregrations

In [84]:


# une requète qui  qui renvoie la quantité totale d'unités vendues cette année (ie tout les mois)

query= """
SELECT*, SUM(quantity) OVER (ORDER BY sale_month) as somme_cumul
FROM sales
"""
db.sql(query)


┌────────────┬────────────┬───────────┬──────────┬──────────────┬──────────┬─────────┬─────────────┐
│  sale_day  │ sale_month │ sale_time │  branch  │   article    │ quantity │ revenue │ somme_cumul │
│  varchar   │  varchar   │  varchar  │ varchar  │   varchar    │  int64   │ double  │   int128    │
├────────────┼────────────┼───────────┼──────────┼──────────────┼──────────┼─────────┼─────────────┤
│ 2021-08-11 │ AUG        │ 11:00     │ New York │ Rolex P1     │        1 │  3000.0 │          14 │
│ 2021-08-14 │ AUG        │ 11:20     │ New York │ Rolex P1     │        2 │  6000.0 │          14 │
│ 2021-08-17 │ AUG        │ 10:00     │ Paris    │ Omega 100    │        3 │  4000.0 │          14 │
│ 2021-08-19 │ AUG        │ 10:00     │ London   │ Omega 100    │        1 │  1300.0 │          14 │
│ 2021-08-01 │ AUG        │ 12:00     │ Paris    │ Tag Heuer T2 │        2 │  5000.0 │          14 │
│ 2021-08-02 │ AUG        │ 14:15     │ London   │ Rolex P1     │        1 │  3200.0 │     

## Over avec les sous clauses PARTITION BY et ORDER BY  pour les fonctions d'aggregrations

In [74]:
# une requète qui  qui renvoie la quantité totale d'unités vendues pour chaque article

query= """
SELECT*, SUM(quantity) OVER(PARTITION BY article ORDER BY sale_month) as with_both
FROM sales
"""
db.sql(query)

┌────────────┬────────────┬───────────┬──────────┬──────────────┬──────────┬─────────┬───────────┐
│  sale_day  │ sale_month │ sale_time │  branch  │   article    │ quantity │ revenue │ with_both │
│  varchar   │  varchar   │  varchar  │ varchar  │   varchar    │  int64   │ double  │  int128   │
├────────────┼────────────┼───────────┼──────────┼──────────────┼──────────┼─────────┼───────────┤
│ 2021-08-17 │ AUG        │ 10:00     │ Paris    │ Omega 100    │        3 │  4000.0 │         7 │
│ 2021-08-19 │ AUG        │ 10:00     │ London   │ Omega 100    │        1 │  1300.0 │         7 │
│ 2021-08-31 │ AUG        │ 16:45     │ Paris    │ Omega 100    │        3 │  4700.0 │         7 │
│ 2021-07-10 │ JUL        │ 11:40     │ London   │ Omega 100    │        2 │  2600.0 │        12 │
│ 2021-07-15 │ JUL        │ 10:30     │ London   │ Omega 100    │        3 │  4000.0 │        12 │
│ 2021-09-20 │ SEP        │ 11:45     │ New York │ Omega 100    │        2 │  5100.0 │        14 │
│ 2021-07-